In [ ]:
# !pip install tf-slim

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import math
import json
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Training NGCF model on all users

In [ ]:
# !python NGCF.py  --adj_type norm --dataset MSD --regs [1e-5] --embed_size 128 --layer_size [128,128,128] --lr 0.0001 --save_flag 1 --pretrain 0 --batch_size 64 --epoch 10 --verbose 1 --node_dropout [0.1] --mess_dropout [0.1,0.1,0.1] --weights_path "./" --proj_path "./"
!python NGCF.py  --alg_type ngcf --adj_type plain --dataset MSD --regs [1e-5] --embed_size 128 --layer_size [128,128,128] --lr 0.0001 --save_flag 1 --pretrain 1 --batch_size 64 --epoch 20 --verbose 1 --node_dropout [0.1] --mess_dropout [0.1,0.1,0.1] --weights_path "./" --proj_path "./"

Generating scores for all user-song pair using the trained model


In [ ]:
!python NGCF.py  --adj_type plain --dataset MSD --regs [1e-5] --embed_size 128 --layer_size [128,128,128] --lr 0.0001 --save_flag 0 --pretrain 1 --batch_size 64 --epoch -1 --verbose 1 --node_dropout [0.1] --mess_dropout [0.1,0.1,0.1] --weights_path "./" --proj_path "./"

Creating test and train files

In [ ]:
triplets = pd.read_csv("/content/drive/MyDrive/Hinsge/filtered_triplets.csv")
triplets.head()

In [ ]:
user_mapping = {}
song_mapping = {}
c=0
for i in sorted(list(set(triplets["user"]))):
    user_mapping[i]=c
    c+=1
c=0
for i in sorted(list(set(triplets["song"]))):
    song_mapping[i]=c
    c+=1

df_to_dict = {}
for ind,row in tqdm(triplets.iterrows(), ncols=100, total=triplets.shape[0]):
    a=user_mapping[row["user"]]
    if a not in df_to_dict :
        df_to_dict[a] = [song_mapping[row["song"]]]
    else:
        df_to_dict[a].append(song_mapping[row["song"]])
    
print(len(df_to_dict))

In [ ]:
with open("user_mapping.json","w") as f:
    json.dump(user_mapping,f)

with open("song_mapping.json","w") as f:
    json.dump(song_mapping,f)

In [ ]:
train = open('train.txt', 'w')
test = open("test.txt","w")
for j in tqdm(sorted(list(df_to_dict.keys())),ncols = 100):  
        k = [str(i) for i in df_to_dict[j]]
        k_train = k[:math.ceil(len(k)*0.8)]
        k_test = k[math.ceil(len(k)*0.8):]
        l = str(j) + " " + " ".join(k_train)+"\n"         
        train.write(l) 
        l = str(j) + " " + " ".join(k_test)+"\n"         
        test.write(l) 
train.close()
test.close()